In [2]:
#unir los archivos en una sola tabla


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10


In [3]:
# 1. EXPLORACIÓN DE LA ESTRUCTURA DE ARCHIVOS
bronze_dir = Path("../data/bronze")
# Listar todos los archivos .parquet
archivos_parquet = list(bronze_dir.glob("*.parquet"))
print(f"\nTotal de archivos encontrados: {len(archivos_parquet)}")

if archivos_parquet:
    print("\n Lista de archivos:")
    for i, archivo in enumerate(archivos_parquet, 1):
        size_mb = archivo.stat().st_size / (1024 * 1024)
        print(f"  {i:2d}. {archivo.name:<30} ({size_mb:.2f} MB)")
else:
    print("  No se encontraron archivos .parquet en la carpeta bronze")


Total de archivos encontrados: 9

 Lista de archivos:
   1. corriente_carga.parquet        (63.19 MB)
   2. humedad_papel.parquet          (0.08 MB)
   3. potencia_aparente.parquet      (4.69 MB)
   4. tap_position.parquet           (0.09 MB)
   5. temperatura_aceite.parquet     (0.17 MB)
   6. temperatura_ambiente.parquet   (0.32 MB)
   7. temperatura_burbujeo.parquet   (0.19 MB)
   8. temperatura_punto_caliente.parquet (0.35 MB)
   9. voltaje.parquet                (44.87 MB)


In [17]:
#analisis exploratorio de la capa bronze para cada uno de los archivos parquet
for archivo in archivos_parquet:
    df = pd.read_parquet(archivo)
    print(f"\nAnálisis del archivo: {archivo.name}")
    print(f"  Número de filas: {len(df)}")
    print(f"  Número de columnas: {len(df.columns)}")
    print(f"  Columnas: {list(df.columns)}")
    print(f"  Tipos de datos:\n{df.dtypes}")
    print(f"  Valores nulos:\n{df.isnull().sum()}")
    print(f"  Estadísticas descriptivas:\n{df.describe()}")
    
#realizar analisis exploratorio de cada uno de los archivos parquet
for archivo in archivos_parquet:
    df = pd.read_parquet(archivo)
    print(f"\nAnálisis del archivo: {archivo.name}")
    print(f"  Número de filas: {len(df)}")
    print(f"  Número de columnas: {len(df.columns)}")
    print(f"  Columnas: {list(df.columns)}")
    print(f"  Tipos de datos:\n{df.dtypes}")
    print(f"  Valores nulos:\n{df.isnull().sum()}")
    print(f"  Estadísticas descriptivas:\n{df.describe()}")



Análisis del archivo: corriente_carga.parquet
  Número de filas: 4419012
  Número de columnas: 2
  Columnas: ['timestamp', 'value']
  Tipos de datos:
timestamp     object
value        float64
dtype: object
  Valores nulos:
timestamp      0
value        940
dtype: int64
  Estadísticas descriptivas:
              value
count  4.418072e+06
mean   7.667462e+02
std    1.833024e+02
min    0.000000e+00
25%    6.364262e+02
50%    7.209121e+02
75%    8.737401e+02
max    1.735917e+03

Análisis del archivo: humedad_papel.parquet
  Número de filas: 5413
  Número de columnas: 2
  Columnas: ['timestamp', 'value']
  Tipos de datos:
timestamp     object
value        float64
dtype: object
  Valores nulos:
timestamp      0
value        904
dtype: int64
  Estadísticas descriptivas:
             value
count  4509.000000
mean      0.805964
std       0.212112
min       0.000000
25%       0.638969
50%       0.800801
75%       0.960834
max       1.497261

Análisis del archivo: potencia_aparente.parquet
  Núm

In [18]:
# Eliminar el código duplicado y mejorar el análisis exploratorio

# ANÁLISIS EXPLORATORIO MEJORADO DE LA CAPA BRONZE
print("🔍 ANÁLISIS EXPLORATORIO DETALLADO - CAPA BRONZE")
print("=" * 60)

def analisis_exploratorio_completo(archivo):
    """
    Realiza un análisis exploratorio completo de un archivo parquet
    """
    print(f"\n📊 ANÁLISIS DETALLADO: {archivo.name}")
    print("-" * 50)
    
    try:
        # Cargar datos
        df = pd.read_parquet(archivo)
        
        # 1. INFORMACIÓN BÁSICA
        print(f"📋 INFORMACIÓN GENERAL:")
        print(f"  • Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
        print(f"  • Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print(f"  • Columnas: {list(df.columns)}")
        
        # 2. TIPOS DE DATOS Y VALORES NULOS
        print(f"\n🔍 CALIDAD DE DATOS:")
        info_columnas = []
        for col in df.columns:
            tipo = str(df[col].dtype)
            nulos = df[col].isnull().sum()
            pct_nulos = (nulos / len(df)) * 100
            unicos = df[col].nunique()
            info_columnas.append({
                'Columna': col,
                'Tipo': tipo,
                'Nulos': nulos,
                '% Nulos': f"{pct_nulos:.1f}%",
                'Únicos': unicos,
                'Cardinalidad': f"{(unicos/len(df)*100):.1f}%"
            })
        
        info_df = pd.DataFrame(info_columnas)
        print(info_df.to_string(index=False))
        
        # 3. ANÁLISIS POR TIPO DE VARIABLE
        cols_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
        cols_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()
        cols_temporales = df.select_dtypes(include=['datetime64']).columns.tolist()
        
        # Variables numéricas
        if cols_numericas:
            print(f"\n📊 VARIABLES NUMÉRICAS ({len(cols_numericas)}):")
            estadisticas = df[cols_numericas].describe().round(3)
            print(estadisticas)
            
            # Detectar posibles outliers usando IQR
            print(f"\n🎯 DETECCIÓN DE OUTLIERS (método IQR):")
            for col in cols_numericas:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                outliers = ((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))).sum()
                pct_outliers = (outliers / len(df)) * 100
                print(f"  • {col}: {outliers} outliers ({pct_outliers:.1f}%)")
        
        # Variables categóricas
        if cols_categoricas:
            print(f"\n📝 VARIABLES CATEGÓRICAS ({len(cols_categoricas)}):")
            for col in cols_categoricas:
                valores_unicos = df[col].nunique()
                valor_mas_frecuente = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else "N/A"
                frecuencia_max = df[col].value_counts().iloc[0] if len(df[col].value_counts()) > 0 else 0
                print(f"  • {col}: {valores_unicos} valores únicos")
                print(f"    - Más frecuente: '{valor_mas_frecuente}' ({frecuencia_max} veces)")
                
                # Mostrar top 5 valores si hay pocos valores únicos
                if valores_unicos <= 10:
                    print(f"    - Distribución: {dict(df[col].value_counts().head())}")
        
        # Variables temporales
        if cols_temporales:
            print(f"\n📅 VARIABLES TEMPORALES ({len(cols_temporales)}):")
            for col in cols_temporales:
                print(f"  • {col}:")
                print(f"    - Rango: {df[col].min()} a {df[col].max()}")
                print(f"    - Duración: {df[col].max() - df[col].min()}")
        
        # 4. ANÁLISIS DE PATRONES ESPECÍFICOS
        print(f"\n🔍 ANÁLISIS DE PATRONES:")
        
        # Buscar columna timestamp
        if 'timestamp' in df.columns:
            print(f"  • Timestamp detectado: {df['timestamp'].dtype}")
            if df['timestamp'].dtype == 'object':
                print(f"    - Ejemplo de formato: {df['timestamp'].iloc[0]}")
                print(f"    - ⚠️ Requiere conversión a datetime")
        
        # Duplicados
        duplicados = df.duplicated().sum()
        print(f"  • Filas duplicadas: {duplicados} ({(duplicados/len(df)*100):.1f}%)")
        
        # Filas completamente vacías
        filas_vacias = df.isnull().all(axis=1).sum()
        print(f"  • Filas completamente vacías: {filas_vacias}")
        
        # 5. RECOMENDACIONES
        print(f"\n💡 RECOMENDACIONES:")
        recomendaciones = []
        
        if duplicados > 0:
            recomendaciones.append(f"Eliminar {duplicados} filas duplicadas")
        
        if 'timestamp' in df.columns and df['timestamp'].dtype == 'object':
            recomendaciones.append("Convertir timestamp a formato datetime")
        
        for col in cols_numericas:
            if df[col].isnull().sum() > 0:
                recomendaciones.append(f"Tratar valores nulos en {col}")
        
        if len(recomendaciones) == 0:
            recomendaciones.append("Dataset en buenas condiciones")
        
        for i, rec in enumerate(recomendaciones, 1):
            print(f"  {i}. {rec}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error al analizar {archivo.name}: {str(e)}")
        return None

# EJECUTAR ANÁLISIS PARA TODOS LOS ARCHIVOS
datasets_analizados = {}
resumen_general = []

for archivo in archivos_parquet:
    df = analisis_exploratorio_completo(archivo)
    if df is not None:
        datasets_analizados[archivo.name] = df
        
        # Recopilar información para resumen general
        resumen_general.append({
            'Archivo': archivo.name,
            'Filas': len(df),
            'Columnas': len(df.columns),
            'Nulos_Total': df.isnull().sum().sum(),
            'Duplicados': df.duplicated().sum(),
            'Memoria_MB': df.memory_usage(deep=True).sum() / 1024**2
        })

# RESUMEN GENERAL
print(f"\n📈 RESUMEN GENERAL DE TODOS LOS DATASETS")
print("=" * 60)

if resumen_general:
    resumen_df = pd.DataFrame(resumen_general)
    print(resumen_df.to_string(index=False))
    
    print(f"\n📊 ESTADÍSTICAS CONSOLIDADAS:")
    print(f"  • Total de archivos analizados: {len(resumen_general)}")
    print(f"  • Filas totales: {resumen_df['Filas'].sum():,}")
    print(f"  • Memoria total: {resumen_df['Memoria_MB'].sum():.2f} MB")
    print(f"  • Promedio de columnas: {resumen_df['Columnas'].mean():.1f}")
    print(f"  • Total de valores nulos: {resumen_df['Nulos_Total'].sum():,}")
    print(f"  • Total de duplicados: {resumen_df['Duplicados'].sum():,}")

print(f"\n✅ Análisis exploratorio completado exitosamente")

🔍 ANÁLISIS EXPLORATORIO DETALLADO - CAPA BRONZE

📊 ANÁLISIS DETALLADO: corriente_carga.parquet
--------------------------------------------------
📋 INFORMACIÓN GENERAL:
  • Dimensiones: 4,419,012 filas × 2 columnas
  • Memoria: 357.65 MB
  • Columnas: ['timestamp', 'value']

🔍 CALIDAD DE DATOS:
  Columna    Tipo  Nulos % Nulos  Únicos Cardinalidad
timestamp  object      0    0.0% 4419011       100.0%
    value float64    940    0.0% 3498513        79.2%

📊 VARIABLES NUMÉRICAS (1):
             value
count  4418072.000
mean       766.746
std        183.302
min          0.000
25%        636.426
50%        720.912
75%        873.740
max       1735.917

🎯 DETECCIÓN DE OUTLIERS (método IQR):
  • value: 73015 outliers (1.7%)

📝 VARIABLES CATEGÓRICAS (1):
  • timestamp: 4419011 valores únicos
    - Más frecuente: '2024-09-19T17:24:34.550003Z' (2 veces)

🔍 ANÁLISIS DE PATRONES:
  • Timestamp detectado: object
    - Ejemplo de formato: 2024-09-10T04:00:02.1890106Z
    - ⚠️ Requiere conversión a